In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
# tf.random.set_seed(123)
# np.random.seed(123)
from sklearn import metrics
import json 
import os

def timeseries_evaluation_metrics_func(y_true, y_pred):
    
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    mse = metrics.mean_squared_error(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print(f'MSE is : {mse}')
    print(f'MAE is : {mae}')
    print(f'RMSE is : {rmse}')
    print(f'MAPE is : {mape}')
    print(f'R2 is : {r2}',end='\n\n')
    return {'mse' : mse, 'mae' : mae, 'rmse' : rmse, 'mape' : mape, 'r2' : r2}

# df = pd.read_csv(r'..\Data\Panama Electricity\continuous_dataset_preprocessing.csv', parse_dates= True)
df = pd.read_csv(
    "../Data/Panama Electricity/continuous_dataset_preprocessing.csv",
    parse_dates=True,
)
df["datetime"] = df["datetime"].apply(pd.to_datetime)

column_names = df.columns.tolist()
train_column_names = df.columns.tolist()
column_names.remove('datetime')
train_column_names.remove('datetime')
train_column_names.remove('nat_demand')

In [3]:
tf.keras.backend.clear_session()
# day_to_test = '1/1/2019'
for month in [*range(1, 12)]:
    day = f'1/{month}/2019'
    valid = df[df['datetime'] <= '1/1/2019 23:00']
    valid = valid[valid['datetime'] > '1/1/2018 23:00']
    train = df[df['datetime'] <= '1/1/2018 23:00']
    test = df[df['datetime'] > day + ' 23:00']
    datatest = df[df['datetime'] <= day + ' 23:00']
    version = 1
    hist_window = 42 # 12, 18, 24, 30, 36, 42, 48, 54, 60
    horizon = 24 # 12, 18, 24, 30, 36
    folder_save = f'../Results/GRU/history_{hist_window}_future_{horizon}_version_{version}'

    model_path = f'{folder_save}/model.h5' 

    x_scaler = preprocessing.MinMaxScaler()
    y_scaler = preprocessing.MinMaxScaler()
    validX = x_scaler.fit_transform(valid[column_names])
    validY = y_scaler.fit_transform(valid[['nat_demand']])

    Trained_model = tf.keras.models.load_model(model_path)
    folder_save = ''

    data_test = x_scaler.fit_transform(datatest[column_names].tail(hist_window))
    test_rescaled = data_test.reshape(1, data_test.shape[0], data_test.shape[1])

    Predicted_results = Trained_model.predict(test_rescaled)
    Predicted_results_Inv_trans = y_scaler.inverse_transform(Predicted_results)
    metrics_test = timeseries_evaluation_metrics_func(test['nat_demand'][:horizon],Predicted_results_Inv_trans[0])

# with open(f"{folder_save}/metrics_test.json", "w") as outfile:
#     json.dump(metrics_test, outfile)

1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 3650.2973486872593
MAE is : 52.13602635091146
RMSE is : 60.4176906931013
MAPE is : 4.635879672183831
R2 is : 0.9111442117401747

1/1 [==============================] - 1s 915ms/step
Evaluation metric results:-
MSE is : 5995.583593784115
MAE is : 59.801669230143226
RMSE is : 77.43115389676248
MAPE is : 5.062179127914524
R2 is : 0.8344138422633229

1/1 [==============================] - 1s 930ms/step
Evaluation metric results:-
MSE is : 13711.847599011475
MAE is : 92.39930034993489
RMSE is : 117.09759860480263
MAPE is : 7.836099703865878
R2 is : 0.6261160396461398

1/1 [==============================] - 1s 867ms/step
Evaluation metric results:-
MSE is : 22193.224427853667
MAE is : 110.65345134277341
RMSE is : 148.97390519098863
MAPE is : 9.840035350590192
R2 is : -0.3703597295573462

1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 1486.4981428447936
MAE is : 33.56